In [1]:
# === WARNINGS ===
# Disable warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
import math
import statistics

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from scipy import signal

mag = 'mag'

def get_magnitude(a): 
    df = pd.read_csv(a)
    
    df[mag] = df.iloc[:, 3]**2 + df.iloc[:, 4]**2 + df.iloc[:, 5]**2
    df[mag] = df[mag].apply(math.sqrt)
    
    return df

fSize = (18, 5)

start = 0
end = 5000

prefix = '191211-december-interim-report-'
ftype = '.png'

save = True

def get_magnitude_image(df, title = None):
    fig = plt.figure(figsize = fSize)
    ax = fig.add_subplot(111)
    ax.plot(df[mag])
    ax.set_title(title)
    ax.set_xlim(start, end)
    plt.show()
    
    if save:
        fig.savefig(prefix + title.lower().replace(' ', '-') + ftype)
    
def get_spectro(df, bins = 70):
    freqs, times, spectro = signal.spectrogram(df[mag], fs=50.0, nperseg=bins, noverlap=bins - 1)
    df = pd.DataFrame({'time':times, 'freq':[freqs[i] for i in np.argmax(spectro,0)]})
    
    return df, spectro

def get_spectro_image(spec, title = None):
    fig = plt.figure(figsize = fSize)
    ax = fig.add_subplot(111)
    ax.imshow(spec, aspect='auto', cmap='hot_r', origin='lower')
    ax.set_title(title)
    ax.set_xlim(start, end)
    plt.show()
    
    if save:
        fig.savefig(prefix + title.lower().replace(' ', '-') + ftype)
    
def get_intensity(spec):
    temp = [list(i) for i in zip(*spec)]
    result = []
    
    for s in range(len(temp)):
        total = 0
        
        for i in range(len(temp[s])):
            val = temp[s][i]
            
            total += val
            
        result.append(total)
        
    return result

def get_spec_flat(sf, title = None, minmax = None):
    fig = plt.figure(figsize = fSize)
    ax = fig.add_subplot(111)
    ax.plot(sf)
    ax.set_title(title)
    ax.set_xlim(start, end)
    
    if minmax:
        ax.plot(minmax['imin'], minmax['vmin'], 'xr')
        ax.plot(minmax['imax'], minmax['vmax'], 'xb')
    
    plt.show()
    
    if save:
        fig.savefig(prefix + title.lower().replace(' ', '-') + ftype)
    
def get_maxima_and_minima(sf, h = None):
    if h is None: 
        h = statistics.mean(np.real(sf))
    
    # Maxima
    i_max = signal.find_peaks(sf, height = h)[0]
    v_max = []
    
    for i in i_max:
        v_max.append(sf[i])
    
    # Minima index
    sf_flipped = np.multiply(sf, -1)
    
    i_min = signal.find_peaks(sf_flipped)[0]
    v_min = []
    
    for i in i_min:
        v_min.append(sf[i])
        
    return {
        'imax': i_max, 
        'vmax': v_max, 
        'imin': i_min, 
        'vmin': v_min
    }

def squish_minmax(mm):
    imin = mm['imin']
    imax = mm['imax']
    vmin = mm['vmin']
    vmax = mm['vmax']
    
    start = 'imin'
    
    if imin[0] > imax[0]:
        start = 'imax'
        
    if start is 'imin':
        fimin = []
        fimax = []
        
        # Counters
        iminc = 0
        imaxc = 0
        
        try:
            while True:
                temp_imin = imin[iminc + 1]

                if temp_imin > imax[imaxc]:
                    if imin[iminc] not in fimin: 
                        fimin.append(imin[iminc])
                    fimax.append(imax[imaxc])
                    imaxc += 1
                    fimin.append(temp_imin)

                iminc += 1
        except IndexError:
            if imax[-1] not in fimax:
                fimax.append(imax[-1]) # Just in case any trailing maxima are not included
    else:
        raise RuntimeError('Unsupported yet')
        
    fvmin = []
    fvmax = []
        
    for v in fimin: 
        for i, val in enumerate(imin):
            if v == val: 
                fvmin.append(vmin[i])
                
    for v in fimax:
        for i, val in enumerate(imax):
            if v == val:
                fvmax.append(vmax[i])
                
    return {
        'imax': fimax,
        'vmax': fvmax,
        'imin': fimin,
        'vmin': fvmin
    }

In [ ]:
path = 'C:\\Users\\liong\\Jupyter Notebook\\191020_Fourth Year Project\\Parkinson-Disease-Repository\\Data\\Patient001-2019-12-05\\'

files = ['ftap-forearm\\ftap-lvl0-forearm_S2_2019-12-05T09.55.32.037_C49948A1895C_Accelerometer_1.4.5.csv',
 'ftap-forearm\\ftap-lvl0-forearm_S2_2019-12-05T09.55.32.037_C49948A1895C_Gyroscope_1.4.5.csv',
 'ftap-wrist\\ftap-lvl0-wrist _S2_2019-12-05T09.44.38.283_C49948A1895C_Accelerometer_1.4.5.csv',
 'ftap-wrist\\ftap-lvl0-wrist _S2_2019-12-05T09.44.38.283_C49948A1895C_Gyroscope_1.4.5.csv']

a = [0, 1, 2, 3]
a = [0, 1]

mag_img_titles = [
    'Accelerometer',
    'Gyroscope'
]

for i in a:
    df = get_magnitude(path + files[i])
    get_magnitude_image(df, title = 'Magnitude of ' + mag_img_titles[i].lower() + ' data')
    
    _, spectro = get_spectro(df)
    get_spectro_image(spectro, title = 'Spectrogram of ' + mag_img_titles[i].lower() + ' data')
    
    if 'Gyroscope' in files[i]:
        spec_flat = get_intensity(spectro)
        get_spec_flat(spec_flat, title = 'Flat spectrogram of ' + mag_img_titles[i].lower() + ' data')
    
        minmax = get_maxima_and_minima(spec_flat)
        get_spec_flat(spec_flat, minmax = minmax, 
                      title = 'Flat spectrogram of ' + mag_img_titles[i].lower() + ' data (max and min)')
        
        minmax = squish_minmax(minmax)
        get_spec_flat(spec_flat, minmax = minmax, 
                      title = 'Flat spectrogram of ' + mag_img_titles[i].lower() + ' data (truncated max and min)')

In [ ]:
files = ['ftap-forearm\\ftap-lvl0-forearm_S2_2019-12-05T09.55.32.037_C49948A1895C_Accelerometer_1.4.5.csv',
 'ftap-forearm\\ftap-lvl0-forearm_S2_2019-12-05T09.55.32.037_C49948A1895C_Gyroscope_1.4.5.csv',
 'ftap-wrist\\ftap-lvl0-wrist _S2_2019-12-05T09.44.38.283_C49948A1895C_Accelerometer_1.4.5.csv',
 'ftap-wrist\\ftap-lvl0-wrist _S2_2019-12-05T09.44.38.283_C49948A1895C_Gyroscope_1.4.5.csv']

a = [3]
mag_img_titles = [
    '',
    '',
    '',
    'second gyroscope'
]

for i in a:
    
    save = False
    
    df = get_magnitude(path + files[i])
    get_magnitude_image(df, title = 'Magnitude of ' + mag_img_titles[i].lower() + ' data')
    
    _, spectro = get_spectro(df)
    get_spectro_image(spectro, title = 'Spectrogram of ' + mag_img_titles[i].lower() + ' data')
    
    if 'Gyroscope' in files[i]:
        spec_flat = get_intensity(spectro)
        get_spec_flat(spec_flat, title = 'Flat spectrogram of ' + mag_img_titles[i].lower() + ' data')
    
        minmax = get_maxima_and_minima(spec_flat)
        get_spec_flat(spec_flat, minmax = minmax, 
                      title = 'Flat spectrogram of ' + mag_img_titles[i].lower() + ' data (max and min)')
        
        save = True
        
        minmax = squish_minmax(minmax)
        get_spec_flat(spec_flat, minmax = minmax, 
                      title = 'Flat spectrogram of ' + mag_img_titles[i].lower() + ' data (truncated max and min)')

In [ ]:
files = [
    'tugt-heel\\tugt-lvl0-heel_S2_2019-12-10T15.27.34.039_C49948A1895C_Gyroscope_1.4.5.csv',
]

a = [0]

mag_img_titles = [
    'third gyroscope'
]

for i in a:
    
    df = get_magnitude(path + files[i])
    get_magnitude_image(df, title = 'Magnitude of ' + mag_img_titles[i].lower() + ' data')
    
    _, spectro = get_spectro(df)
    get_spectro_image(spectro, title = 'Spectrogram of ' + mag_img_titles[i].lower() + ' data')
    
    if 'Gyroscope' in files[i]:
        spec_flat = get_intensity(spectro)
        get_spec_flat(spec_flat, title = 'Flat spectrogram of ' + mag_img_titles[i].lower() + ' data')
    
        save = False
    
        minmax = get_maxima_and_minima(spec_flat)
        get_spec_flat(spec_flat, minmax = minmax, 
                      title = 'Flat spectrogram of ' + mag_img_titles[i].lower() + ' data (max and min)')
        
        save = True
        
        minmax = squish_minmax(minmax)
        get_spec_flat(spec_flat, minmax = minmax, 
                      title = 'Flat spectrogram of ' + mag_img_titles[i].lower() + ' data (truncated max and min)')

In [ ]:
True